In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Canavan_Disease"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Canavan_Disease/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Canavan_Disease/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Canavan_Disease/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Canavan_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Canavan Disease
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Search for related terms to Canavan Disease
related_terms = ["canavan", "leukodystrophy", "aspa", "neurological", "brain", "cns", "central nervous system", "glioma"]
matched_dir = None

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    # Check if any related term is in the directory name
    if any(term in subdir_lower for term in related_terms):
        matched_dir = subdir
        print(f"Found potential match: {subdir}")
        # Prioritize more specific matches
        if "glioma" in subdir_lower or "gbm" in subdir_lower:
            print(f"Selected as best match: {subdir}")
            matched_dir = subdir
            break

# If we found a potential match, use it
if matched_dir:
    print(f"Selected directory: {matched_dir}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, matched_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not matched_dir:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Lower_Grade_Glioma_(LGG)
Selected as best match: TCGA_Lower_Grade_Glioma_(LGG)
Selected directory: TCGA_Lower_Grade_Glioma_(LGG)
Clinical file: TCGA.LGG.sampleMap_LGG_clinicalMatrix
Genetic file: TCGA.LGG.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_performance_status_assessment', 'eastern_cancer_oncology_group', 'eczema_history', 'family_history_of_cancer', 'family_history_of_primary_brain_tumor', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 
                     'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy']
candidate_gender_cols = ['gender']

# Read the clinical data file
cohort_dir = os.path.join(tcga_root_dir, "TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)")
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_animal_insect_allergy': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan]}

Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspecting the age columns
age_columns = {
    'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 
    'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 
    'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
    'first_diagnosis_age_of_animal_insect_allergy': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')], 
    'first_diagnosis_age_of_food_allergy': [float('nan'), float('nan'), float('nan'), float('nan'), float('nan')]
}

# Inspecting the gender columns
gender_columns = {
    'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']
}

# Select the best age column
# 'age_at_initial_pathologic_diagnosis' has actual age values in years (preferred)
# 'days_to_birth' has negative values representing days since birth, which would need conversion
# The other columns have all NaN values

age_col = 'age_at_initial_pathologic_diagnosis'

# Select the best gender column
# Only one column is available and it has valid values
gender_col = 'gender'

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Age column preview: {age_columns[age_col]}")
print(f"Selected gender column: {gender_col}")
print(f"Gender column preview: {gender_columns[gender_col]}")


Selected age column: age_at_initial_pathologic_diagnosis
Age column preview: [44.0, 50.0, 59.0, 56.0, 40.0]
Selected gender column: gender
Gender column preview: ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Canavan_Disease/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 530)
Clinical data saved to ../../output/preprocess/Canavan_Disease/clinical_data/TCGA.csv
Clinical data shape: (1148, 3)
Number of samples in clinical data: 1148
Number of samples in genetic data: 530
Number of common samples: 530
Linked data shape: (530, 19851)


Data shape after handling missing values: (530, 19851)
Quartiles for 'Canavan_Disease':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Canavan_Disease' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 32.0
  50% (Median): 41.0
  75%: 53.0
Min: 14.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 238 occurrences. This represents 44.91% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable based on validation criteria. Data not saved.
Preprocessing completed.
